In [1]:
#import packages
import requests
import json
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
path = "Park Unit Scraping Information.csv"
regionlookup = pd.read_csv(path, encoding='latin-1')
regionlookup.head()

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,Alagnak,Alagnak Wild River,Wild River,AK,AKR,ALAG,1
1,Aniakchak,Aniakchak National Monument and Preserve,National Monument & Preserve,AK,AKR,ANIA,2
2,Bering Land Bridge,Bering Land Bridge National Preserve,National Preserve,AK,AKR,BELA,1
3,Cape Krusenstern,Cape Krusenstern National Monument,National Monument,AK,AKR,CAKR,1
4,Denali,Denali National Park and Preserve,National Park & Preserve,AK,AKR,DENA,2


In [3]:
regionlookup[regionlookup['Alpha'] == 'DEVA']

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
429,Death Valley,Death Valley National Park,National Park,"CA, NV",PWR,DEVA,1


In [4]:
regionlookup.columns

Index(['Park Name', 'Park Full Name', 'Type of Unit', 'State', 'Reg', 'Alpha',
       'Unit'],
      dtype='object')

In [5]:
#pull API using API key
response_API = requests.get('https://developer.nps.gov/api/v1/roadevents?api_key=3AidKP0PiqgfP8sbwZC0RZ9JM6QDnvYaZx3RRjrH')
#print(response_API.status_code)
data = response_API.text
df = json.loads(data)

In [6]:
#summarize the data by park
park_list = []
for x in df["features"]:
    for y in df["road_event_feed_info"]['data_sources']:
        if y['data_source_id']==x['properties']['core_details']['data_source_id']:
            park_list.append(y['organization_name'])

park_df = Counter(park_list)
park_df = pd.DataFrame.from_dict(park_df, orient="index").reset_index().rename(
    columns={'index': 'park',
            0:'number of alerts'}).sort_values(by=['number of alerts'])


In [7]:
# Get today's date as a string
today = datetime.date.today()
today_string = today.strftime('%Y-%m-%d')

park_df['date'] = today_string

In [8]:
len(park_df), sum(park_df['number of alerts']), max(park_df['number of alerts'])

(13, 58, 20)

In [9]:
# Loop through the API entries
counter=0

park_list = []
alert_name_list = []
description_list = []
event_type_list = []
road_name_list = []
vehicle_impact_list = []
start_date_list = []
end_date_list = []
today_date_list = []
id_list = []


for x in df["features"]:
    for y in df["road_event_feed_info"]['data_sources']:
        if y['data_source_id']==x['properties']['core_details']['data_source_id']:
            print("Park:", y['organization_name'])
            park_list.append(y['organization_name'])
    print("Alert name: ", x['properties']['core_details']['name'])
    print("Description: ", x['properties']['core_details']['description'])
    print("Event type: ", x['properties']['core_details']['event_type'])
    print("Road name: ", x['properties']['core_details']['road_names'])
    print("Vehicle impact: ", x['properties']['vehicle_impact'])
    print("Alert ID: ", x['properties']['core_details']['data_source_id'])
  #  print("Dates: ", x['properties']['start_date'], "to ", x['properties']['end_date'])
    print("")
    alert_name_list.append(x['properties']['core_details']['name'])
    description_list.append(x['properties']['core_details']['description'])
    event_type_list.append(x['properties']['core_details']['event_type'])
    road_name_list.append(x['properties']['core_details']['road_names'])
    vehicle_impact_list.append(x['properties']['vehicle_impact'])
    start_date_list.append(x['properties']['start_date'])
    id_list.append(x['properties']['core_details']['data_source_id'])
   # end_date_list.append(x['properties']['end_date'])
    today_date_list.append(today_string)
    counter+=1

Park: Crater Lake National Park
Alert name:  Ease Rim Drive Rehabilitation
Description:  East Rim Drive is closed from Skell Head to Phantom Ship Overlook for Year 2 of a major rehabilitation project.  Cloudcap Road is also closed. 
Event type:  work-zone
Road name:  ['East Rim Drive from Skell Head to Phantom Ship Overlook']
Vehicle impact:  all-lanes-closed
Alert ID:  02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2

Park: Crater Lake National Park
Alert name:  Ease Rim Drive Rehabilitation
Description:  East Rim Drive is closed from Skell Head to Phantom Ship Overlook for Year 2 of a major rehabilitation project.  Cloudcap Road is also closed. 
Event type:  work-zone
Road name:  ['East Rim Drive from Skell Head to Phantom Ship Overlook']
Vehicle impact:  all-lanes-closed
Alert ID:  02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2

Park: Crater Lake National Park
Alert name:  Ease Rim Drive Rehabilitation
Description:  East Rim Drive is closed from Skell Head to Phantom Ship Overlook for Year 2 of a major r

In [10]:
df_update = pd.DataFrame({
    "date":today_date_list,
    "park":park_list,
    "alert name": alert_name_list,
    "description": description_list,
    "event type": event_type_list,
    "road name": road_name_list,
    "vehicle impact": vehicle_impact_list,
    "event start date": start_date_list,
    "alert_id": id_list
#    "event end date": end_date_list
})
df_update.head()

,date,park,alert name,description,event type,road name,vehicle impact,event start date,alert_id
0,2024-08-22,Crater Lake National Park,Ease Rim Drive Rehabilitation,East Rim Drive is closed from Skell Head to Ph...,work-zone,[East Rim Drive from Skell Head to Phantom Shi...,all-lanes-closed,2024-07-02T03:58:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2
1,2024-08-22,Crater Lake National Park,Ease Rim Drive Rehabilitation,East Rim Drive is closed from Skell Head to Ph...,work-zone,[East Rim Drive from Skell Head to Phantom Shi...,all-lanes-closed,2024-07-02T03:58:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2
2,2024-08-22,Crater Lake National Park,Ease Rim Drive Rehabilitation,East Rim Drive is closed from Skell Head to Ph...,work-zone,[East Rim Drive from Skell Head to Phantom Shi...,all-lanes-closed,2024-07-02T03:58:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2
3,2024-08-22,Crater Lake National Park,Ease Rim Drive Rehabilitation,East Rim Drive is closed from Skell Head to Ph...,work-zone,[East Rim Drive from Skell Head to Phantom Shi...,all-lanes-closed,2024-07-02T03:58:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2
4,2024-08-22,Crater Lake National Park,North Entrance Road Temporary Closure,North Entrance Road is temporarily closed whil...,incident,[North Entrance Road],all-lanes-closed,2024-08-13T12:30:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2


In [11]:
regionlookup.rename(columns={'Park Full Name':"park", "Reg":"region","Alpha":"alpha"}, inplace = True)
region_subset = regionlookup[['park', 'region', 'alpha', "State"]]
region_subset.head()

,park,region,alpha,State
0,Alagnak Wild River,AKR,ALAG,AK
1,Aniakchak National Monument and Preserve,AKR,ANIA,AK
2,Bering Land Bridge National Preserve,AKR,BELA,AK
3,Cape Krusenstern National Monument,AKR,CAKR,AK
4,Denali National Park and Preserve,AKR,DENA,AK


In [12]:
merged_df = pd.merge(df_update, region_subset, on="park", how="inner")
merged_df.head()

,date,park,alert name,description,event type,road name,vehicle impact,event start date,alert_id,region,alpha,State
0,2024-08-22,Crater Lake National Park,Ease Rim Drive Rehabilitation,East Rim Drive is closed from Skell Head to Ph...,work-zone,[East Rim Drive from Skell Head to Phantom Shi...,all-lanes-closed,2024-07-02T03:58:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
1,2024-08-22,Crater Lake National Park,Ease Rim Drive Rehabilitation,East Rim Drive is closed from Skell Head to Ph...,work-zone,[East Rim Drive from Skell Head to Phantom Shi...,all-lanes-closed,2024-07-02T03:58:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
2,2024-08-22,Crater Lake National Park,Ease Rim Drive Rehabilitation,East Rim Drive is closed from Skell Head to Ph...,work-zone,[East Rim Drive from Skell Head to Phantom Shi...,all-lanes-closed,2024-07-02T03:58:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
3,2024-08-22,Crater Lake National Park,Ease Rim Drive Rehabilitation,East Rim Drive is closed from Skell Head to Ph...,work-zone,[East Rim Drive from Skell Head to Phantom Shi...,all-lanes-closed,2024-07-02T03:58:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR
4,2024-08-22,Crater Lake National Park,North Entrance Road Temporary Closure,North Entrance Road is temporarily closed whil...,incident,[North Entrance Road],all-lanes-closed,2024-08-13T12:30:00Z,02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2,PWR,CRLA,OR


In [13]:
df_old_update = pd.read_csv("./TDx_tracker.csv")

In [14]:
df_update = pd.concat([merged_df, df_old_update])
df_update['date'] = pd.to_datetime(df_update.date)


In [15]:
null_region_count = merged_df['region'].isnull().sum()
null_region_count

0

In [16]:
if null_region_count == 0:
    df_update.to_csv("./TDx_tracker.csv", index=False)
else:
    null_region_rows = joined[joined['region'].isnull()]
    print("Rows with a null Region column:")
    print(null_region_rows)

In [17]:
path = "Park Unit Scraping Information.csv"
parks = pd.read_csv(path, encoding='latin-1')
parks.head()

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,Alagnak,Alagnak Wild River,Wild River,AK,AKR,ALAG,1
1,Aniakchak,Aniakchak National Monument and Preserve,National Monument & Preserve,AK,AKR,ANIA,2
2,Bering Land Bridge,Bering Land Bridge National Preserve,National Preserve,AK,AKR,BELA,1
3,Cape Krusenstern,Cape Krusenstern National Monument,National Monument,AK,AKR,CAKR,1
4,Denali,Denali National Park and Preserve,National Park & Preserve,AK,AKR,DENA,2


In [18]:
df_update.park.value_counts()

park
Blue Ridge Parkway                            392
Crater Lake National Park                     390
Mojave National Preserve                      223
Great Smoky Mountains National Park           220
Shenandoah National Park                      166
Yosemite National Park                        150
Capitol Reef National Park                     92
Buffalo National River                         90
Curecanti National Recreation Area             84
Organ Pipe Cactus National Monument            78
Death Valley National Park                     78
Guadalupe Mountains National Park              78
Ozark National Scenic Riverways                74
Black Canyon of the Gunnison National Park     74
Sleeping Bear Dunes National Lakeshore         59
Zion National Park                             44
De Soto National Memorial                      40
Redwood National Park                          29
Natchez Trace Parkway                          24
Dinosaur National Monument                   

In [19]:
len(df_update.park.unique())

32

In [20]:
running_park_list = []
date_list = []
park_count_list = []

df_update = df_update.sort_values(by=['date'])

for x in df_update.date.unique():
    df_date = df_update.loc[df_update['date']==x]
    print(x, ": ", len(df_date))
    for y in df_date['park']:
        if y not in running_park_list:
            print(y)
            running_park_list.append(y)
    date_list.append(x)
    park_count_list.append(len(running_park_list))

2023-10-17 00:00:00 :  11
Yosemite National Park
Mojave National Preserve
Acadia National Park
Dinosaur National Monument
Natchez Trace Parkway
Olympic National Park
Golden Gate National Recreation Area
2023-10-23 00:00:00 :  11
2023-11-27 00:00:00 :  4
Padre Island National Seashore
2023-12-18 00:00:00 :  4
Muir Woods National Monument
Fort Point National Historic Site
2023-12-28 00:00:00 :  3
2024-01-04 00:00:00 :  3
2024-01-11 00:00:00 :  5
Shenandoah National Park
2024-01-18 00:00:00 :  4
2024-01-25 00:00:00 :  3
2024-02-01 00:00:00 :  8
2024-02-05 00:00:00 :  61
Organ Pipe Cactus National Monument
Death Valley National Park
Bryce Canyon National Park
Valley Forge National Historical Park
Guadalupe Mountains National Park
Zion National Park
Redwood National Park
Point Reyes National Seashore
Crater Lake National Park
2024-03-04 00:00:00 :  83
Buffalo National River
Great Smoky Mountains National Park
Piscataway Park
2024-04-02 00:00:00 :  62
Ozark National Scenic Riverways
Black Ca

In [21]:
df_count = pd.DataFrame({
    'date': date_list,
    "count of parks": park_count_list
})
df_count['date'] = pd.to_datetime(df_count['date'])


In [54]:
date_list

[Timestamp('2023-10-17 00:00:00'),
 Timestamp('2023-10-23 00:00:00'),
 Timestamp('2023-11-27 00:00:00'),
 Timestamp('2023-12-18 00:00:00'),
 Timestamp('2023-12-28 00:00:00'),
 Timestamp('2024-01-04 00:00:00'),
 Timestamp('2024-01-11 00:00:00'),
 Timestamp('2024-01-18 00:00:00'),
 Timestamp('2024-01-25 00:00:00'),
 Timestamp('2024-02-01 00:00:00'),
 Timestamp('2024-02-05 00:00:00'),
 Timestamp('2024-03-04 00:00:00'),
 Timestamp('2024-04-02 00:00:00'),
 Timestamp('2024-04-19 00:00:00'),
 Timestamp('2024-04-26 00:00:00'),
 Timestamp('2024-04-29 00:00:00'),
 Timestamp('2024-05-02 00:00:00'),
 Timestamp('2024-05-08 00:00:00'),
 Timestamp('2024-05-22 00:00:00'),
 Timestamp('2024-06-06 00:00:00'),
 Timestamp('2024-06-17 00:00:00'),
 Timestamp('2024-06-28 00:00:00'),
 Timestamp('2024-07-02 00:00:00'),
 Timestamp('2024-07-03 00:00:00'),
 Timestamp('2024-07-10 00:00:00'),
 Timestamp('2024-07-16 00:00:00'),
 Timestamp('2024-07-23 00:00:00'),
 Timestamp('2024-07-24 00:00:00'),
 Timestamp('2024-07-

In [55]:
# Adjusting the figure size
fig, ax = plt.subplots(1,1,figsize=(20,10))

# Creating a plot
plt.plot(df_count.date, df_count['count of parks'], linewidth = 3)

# Adding a plot title and customizing its font size
plt.title('Running Count of Parks that have Created TDx Alerts (248 Parks with Roads)', fontsize=20)

bar_max = df_count['count of parks']*1.1
plt.text(365, bar_max,
           'Asheville,NC\nMP 370-400', fontsize=15)
plt.vlines([100, 130, 280, 300, 370,400], ymin=0, ymax = bar_max/1.05,
            linestyles = "dotted", linewidth = 2)


# Adding axis labels and customizing their font size
plt.xlabel('Date', fontsize=15)
plt.ylabel('Count of Parks', fontsize=15)

# Rotaing axis ticks and customizing their font size
plt.xticks(rotation=30, fontsize=15)
plt.ylim([0,max(df_count['count of parks'])*1.25])

# Changing the plot resolution - zooming in the period from 15.12.2022 till 15.01.2023
#plt.xlim(pd.Timestamp('2023-09-1'), pd.Timestamp('2024-02-15'))
plt.plot()


ValueError: could not broadcast input array from shape (35,) into shape (6,)

Error in callback <function _draw_all_if_interactive at 0x0000016F065FD8B0> (for post_execute):


TypeError: cannot convert the series to <class 'float'>

TypeError: cannot convert the series to <class 'float'>

<Figure size 2000x1000 with 1 Axes>

In [23]:
df_count

,date,count of parks
0,2023-10-17,7
1,2023-10-23,7
2,2023-11-27,8
3,2023-12-18,10
4,2023-12-28,10
5,2024-01-04,10
6,2024-01-11,11
7,2024-01-18,11
8,2024-01-25,11
9,2024-02-01,11


## Alerts

In [35]:
#pull API using API key
response_API = requests.get('https://developer.nps.gov/api/v1/alerts?limit=1000&api_key=3AidKP0PiqgfP8sbwZC0RZ9JM6QDnvYaZx3RRjrH')
#print(response_API.status_code)
data = response_API.text
df = json.loads(data)

In [36]:
id_list = []
url_list = []
title_list = []
parkCode_list = []
description_list = []
category_list = []
relatedRoadEvents_list = []
lastIndexedDate_list = []

for x in df['data']:
    #print(x)
    id_list.append(x['id'])
    url_list.append(x['url']) 
    title_list.append(x['title']) 
    parkCode_list.append(x['parkCode'].upper()) 
    description_list.append(x['description'])
    category_list.append(x['category']) 
    relatedRoadEvents_list.append(x['relatedRoadEvents'])
    lastIndexedDate_list.append(x['lastIndexedDate'])


In [37]:
alerts_df = pd.DataFrame({
    'id': id_list, 
    'url': url_list, 
    'title': title_list,
    'park alpha': parkCode_list,
    'description': description_list,
    'category': category_list,
    'relatedRoadEvents':relatedRoadEvents_list,
    'LastIndexedDate':lastIndexedDate_list
})

In [38]:
alerts_df2 = alerts_df.merge(parks,left_on='park alpha', right_on="Alpha", how = "left")

In [39]:
road_closed_flag_list = []
road_words_list = ["road", "route", "interstate", "highway", "vehic"]

for x in range(len(alerts_df2)): 
    road_closed_flag = "No"
    
    for y in road_words_list:
        if y in str(alerts_df2["description"][x].lower()):
            road_closed_flag = "Yes"
            break
            
    road_closed_flag_list.append(road_closed_flag)

alerts_df2["Road Closed Flag"] = road_closed_flag_list

In [40]:
# add to alerts tracker table
alerts_df2['date'] = today_string
alerts_df2_old = pd.read_csv("./alerts_api_tracker.csv")
alerts_df2_combined = pd.concat([alerts_df2, alerts_df2_old])
alerts_df2_combined.to_csv("alerts_api_tracker.csv", index=False)

##### Create a tracking table

NPS wants to know how many total road closures there are, and how many of these are included in the TDx feed. For this, we can subset our alerts table for road closure alerts then compare our 2 tables. 

In [41]:
# filter our alerts table to only include road closures
alerts_df3 = alerts_df2.loc[alerts_df2['Road Closed Flag']=="Yes"]

# group by our park-related fields and create sub-lists for our alert info
alerts_df3_grouped = alerts_df3.groupby(['park alpha','Park Name', 'Park Full Name',
       'Type of Unit', 'State','Reg'])[['title', 'description', 'category',
       'relatedRoadEvents']].agg(lambda x: list(x)).reset_index()

In [42]:
# create list of the unique park alphas in our two tables
has_alerts = set(alerts_df3['park alpha'])
has_tdx = set(merged_df['alpha'])

# create sublists of overlapping alphas, only in alerts table, only in TDx table
has_both = has_alerts.intersection(has_tdx)
only_alerts = has_alerts - has_tdx
only_tdx = has_tdx - has_alerts

#create empty data frame then populate the rows
status = pd.DataFrame(columns=['Park Alpha', 'Status'])
status['Park Alpha'] = list(has_both) + list(only_alerts) + list(only_tdx)
status['Status'] = (['Using TDx feed and alerts'] * len(has_both)) + (['Only using alerts'] * len(only_alerts)) + (['Only using TDx feed'] * len(only_tdx))

In [43]:
# join our status table with our alerts table
status_merged = status.merge(alerts_df3_grouped,left_on='Park Alpha', right_on="park alpha", how = "left")

In [44]:
status_merged['Status'].value_counts()

Status
Only using alerts            97
Using TDx feed and alerts    12
Name: count, dtype: int64

In [45]:
status_merged.to_csv("combined_status.csv", index=False)

#### Create a tracker spreadsheet with the combined updates

In [46]:
df_update_combined = status_merged
df_update_combined['date'] = today_string

In [47]:
df_old_update = pd.read_csv("./combined_status_tracker.csv")

In [48]:
df_update_combined = pd.concat([df_update_combined, df_old_update])

In [49]:
df_update_combined.to_csv("combined_status_tracker.csv", index=False)

#### Calculate visitors going to a park with TDx out of all visitors going to a park with a road closure, using hardcoded 2014-2023 data

In [50]:
#Recreation visitor data for 2014-2023, inclusive, downloaded from NPS IRMA Public Use Statistics QueryBuilder, saved in the parent folder.
#If this file is changed (e.g. new download with future years' data), delete rows 1 and 2 in the file. The first row should be the column headings.
df_10year_visits = pd.read_excel("./2014_2023_Visitation_All_Parks.xlsx")

df_10year_visits.columns = df_10year_visits.columns.str.strip()
df_10year_visits.sort_values(by=['Unit Code'], ascending=True)

df_10year_visits['TenAvg'] = df_10year_visits.groupby(by='Unit Code')['Recreation Visits'].transform('mean')

#Subset of all parks: visits at parks using TDx or TDx and alerts or Alerts only (i.e. all road closures)
df_10year_visits_at_allroadclosure_parks = df_10year_visits[df_10year_visits['Unit Code'].isin(has_both|only_tdx|only_alerts)]

#Number of visits at parks with road closures
AllRoadClosureTotal = df_10year_visits_at_allroadclosure_parks['Recreation Visits'].sum()
print("10 year average visits to parks with a current road closure:", format(AllRoadClosureTotal,','))

#Subset of all parks: visits at parks using TDx or TDx and alerts
df_10year_visits_at_TDx_parks = df_10year_visits[df_10year_visits['Unit Code'].isin(has_both|only_tdx)]

#Number of visits at parks with TDx
ParksUsingTDxTotal = df_10year_visits_at_TDx_parks['Recreation Visits'].sum()
print("10 year visits to parks using TDx for an active road closure:", format(ParksUsingTDxTotal,','))
print("This is",(round((ParksUsingTDxTotal/AllRoadClosureTotal)*100)),"% of 10 year average visits.")

10 year average visits to parks with a current road closure: 1,286,183,933
10 year visits to parks using TDx for an active road closure: 348,707,645
This is 27 % of 10 year average visits.


In [45]:
#### Basic Keyword search

In [46]:
#### To do: Create list of keywords
sample_alerts_df = alerts_df3.description.str.split(expand=True).stack().value_counts()
print(sample_alerts_df.head(45))

the             281
to              222
and             167
is              110
of               89
in               79
for              76
on               62
at               61
will             57
Road             55
are              52
The              50
a                50
road             50
closed           48
park             47
be               47
from             46
or               40
your             39
parking          38
may              31
all              31
not              29
due              26
vehicle          26
with             23
vehicles         23
construction     22
Trail            21
open             21
Please           20
can              20
access           18
along            18
roads            18
Visitor          16
closure          16
Due              16
&                14
that             14
traffic          14
entrance         14
you              14
Name: count, dtype: int64
